In [2]:
import pandas as pd

# Vancouver Crime data

In [22]:
vancouver_data_path = '../data/vancouver_crime_data/'

In [28]:
vancouver_crimes = pd.read_csv(vancouver_data_path + 'crimedata_csv_all_years.csv')

## Convert X Y to Longitude Latitude

Using https://webapp.geod.nrcan.gc.ca/geod/tools-outils/trx.php

In [29]:
# Set format for tool
vancouver_crimes.rename(columns = {'X': 'utm_e', 'Y': 'utm_n'}, inplace = True)
vancouver_crimes['height'] = 0
vancouver_crimes['utm_z'] = 'utm10'

# Save
vancouver_crimes.to_csv(vancouver_data_path + 'crime_data_prep.csv', index = False)

Upload crime_data_prep file to site with settings UTM/MTM/Sterero UTM10 to Geographic. 

In [30]:
# Add longitude and latitude and reformat
geographic_data = pd.read_csv(vancouver_data_path + 'TRX_2020-02-18_52723.csv')
vancouver_crimes.rename(columns = {'utm_e': 'X', 'utm_n': 'Y'}, inplace = True)
vancouver_crimes.drop(['height', 'utm_z'], axis = 1, inplace = True)
vancouver_crimes[['Latitude', 'Longitude']]= geographic_data[['lat', 'lon']]

# Save 
vancouver_crimes.to_csv(vancouver_data_path + 'crime_data.csv', index = False)